In [ ]:
import numpy as np
import librosa
import json
import tensorflow as tf
import keras
#from keras import backend as K


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FILES_PATH = "/content/drive/MyDrive/Colab Notebooks/files/"
#FILES_PATH = "/content/drive/MyDrive/"

In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/files

ebow-whatsapp-mic-recording-test-cut-44100.wav	saved_model.hdf5  zero_encoded_classes.json


In [ ]:
#!pip install librosa
#import librosa
from IPython.display import Audio
from matplotlib import pyplot as plt
import seaborn as sns

# all imports
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment
import numpy as np
import time

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Recording...")
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  audio = AudioSegment.from_file(BytesIO(b))
  return audio


# Loading the pre-trained model

---



In [ ]:
# LOAD model
h5_file = FILES_PATH+"saved_model.hdf5"
loaded_model = tf.keras.models.load_model(h5_file)

# Load from the JSON file
with open(FILES_PATH+'zero_encoded_classes.json', 'r') as json_file:
    loaded_zero_encoded_classes = json.load(json_file)

print(loaded_zero_encoded_classes)


['slide', 'imanes', 'ebow', 'arco', 'cenicero']


In [ ]:
n_fft = 2048
hop_length = 4*256

In [ ]:
def preprocess_input(example_filename, sample_rate=None, audio_duration=None):
    # Open file with original sample rate
    example_audio, orig_sr = librosa.load(example_filename, sr=sample_rate)

    if audio_duration is not None:
        stop = int(audio_duration*orig_sr)
        X_truncated = example_audio[0:stop]
        X_truncated = np.hstack([X_truncated, np.zeros(stop-len(X_truncated))])
    else:
        X_truncated = example_audio

    mel_spec = librosa.feature.melspectrogram(y=X_truncated, sr=sample_rate, hop_length=hop_length, n_fft=n_fft)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    image_input = mel_spec_db.reshape(*mel_spec_db.shape , 1)
    image_input = np.repeat(image_input, 3, axis=-1)
    image_input = np.expand_dims(image_input, axis=0)
    # image.shape

    return X_truncated, orig_sr, mel_spec_db, image_input

In [ ]:
def decode_one_hot(encoded_output, class_labels):
    # Find the index of the highest value in the one-hot encoded array
    index = np.argmax(encoded_output)

    # Map the index to the corresponding class label
    class_label = class_labels[index]

    return class_label

# Realtime prediction

In [ ]:
sample_rate = 44100

def capturar_predecir(records_amount=5):
  audio_duration = 3
  audio=record(sec=audio_duration) # 3 segundos

  #evaluar si realmente es necesario escribir el archivo primero
  sound = "test"
  nbr = 1
  example_filename = "{}_{}.wav".format(sound,nbr)
  audio.export(example_filename,format="wav")

  #TODO: check sample rate
  _, _, _, image_input = preprocess_input(example_filename, sample_rate=sample_rate, audio_duration=audio_duration)
  predictions = loaded_model.predict( image_input ) # %time
  print( predictions )
  print( "Clases posibles: ", loaded_zero_encoded_classes)
  print( "Clase detectada: ", decode_one_hot(predictions[0], loaded_zero_encoded_classes) )
  print( "--------------------------------------------")
  print( "--------------------------------------------")
  time.sleep(1)

In [ ]:
for i in range(5):
  capturar_predecir()

Recording...


<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 122ms/step
[[0.5980387  0.05296788 0.15838645 0.1234619  0.06714505]]
Clases posibles:  ['slide', 'imanes', 'ebow', 'arco', 'cenicero']
Clase detectada:  slide
--------------------------------------------
--------------------------------------------
Recording...


<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 119ms/step
[[0.06758568 0.03572444 0.7860498  0.10108009 0.00956001]]
Clases posibles:  ['slide', 'imanes', 'ebow', 'arco', 'cenicero']
Clase detectada:  ebow
--------------------------------------------
--------------------------------------------
Recording...


...

1/1 [==============================] - 0s 156ms/step
[[0.08854766 0.09686952 0.5762151  0.14109325 0.09727445]]
Clases posibles:  ['slide', 'imanes', 'ebow', 'arco', 'cenicero']
Clase detectada:  ebow
--------------------------------------------
--------------------------------------------
Recording...


<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 147ms/step
[[2.7300960e-01 7.2570306e-01 5.3549214e-04 6.6796801e-04 8.3803658e-05]]
Clases posibles:  ['slide', 'imanes', 'ebow', 'arco', 'cenicero']
Clase detectada:  imanes
--------------------------------------------
--------------------------------------------
Recording...


<IPython.core.display.Javascript object>

1/1 [==============================] - 0s 141ms/step
[[0.86326027 0.02020462 0.06473632 0.04729205 0.00450683]]
Clases posibles:  ['slide', 'imanes', 'ebow', 'arco', 'cenicero']
Clase detectada:  slide
--------------------------------------------
--------------------------------------------


In [ ]:
# Paso 1) enviar MIDI
# Paso 2) detectar cosas y cambiar parámetros (ej en base a duración)
# Control / Recomendación